# R/C Implementation of a Scalar Bekk Model

## Introduction

## Implementation Notes
### Parameter convention:
The model uses two parameters:
* $\alpha$: the factor of the lagged $y^2$
* $\beta$: the factor of the lagged sigma
However, since the optimisation libraries only support box constraints, a trick must be used to keep not only $\alpha$ and $\beta$ between 0 and 1, but also $0 \leq \alpha+\beta \leq 1$. The way to do this, is to feed two parameters into the filter, $\gamma$ and $\lambda$ such that:
$$
\alpha = (1-\lambda) * \gamma \\
\beta = (1-\lambda) * (1 - \gamma)
$$
and then we constrain these two parameters to $(0,1)$. This results in the overall step for an update looking as follows:
$$
S_t = \lambda C + \sum_{k=1}^K \frac{\alpha}{k} y_{t-k}y_{t-k}^{T} + \frac{\beta}{k} S_{t-k}
$$

### Sigma Initialisation
$\S_0$ is initialized at the unconditional covariance, so is $C$. Both are then cholesky decomposed for the actual updates.

## Examples

In [1]:
dyn.load('./C/bekk_log_lik.so')
source('./R/bekk_model.R')

In [2]:
y <- read.csv('../ganter/data/Problemsets Data (Tickers)/all.csv')
y <- apply(y,c(2),diff)
y[is.na(y)] <- 0

Calculating a single filter pass with given parameters

In [3]:
scalar.bekk.filter(as.matrix(y[1:10,2:4]),c(0.9,0.5),k=1)

$Sig
 [1]  4.957518e-01  4.463798e-01  3.942723e-01  3.432869e-01  3.025566e-01
 [6]  2.724126e-01  2.821938e-01  2.626694e-01  2.948682e-01  2.675401e-01
[11]  3.026712e-01  2.942328e-01  2.353373e-01  2.065666e-01  1.683365e-01
[16]  1.435516e-01  1.799001e-01  1.538633e-01  1.436962e-01  1.163869e-01
[21]  1.700503e-02  6.369785e-03  9.097835e-04  1.033304e-02  7.265820e-03
[26]  5.570361e-03  7.432129e-05  2.933433e-03 -1.351978e-02 -1.238393e-02
[31]  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
[36]  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
[41]  4.399891e-01  3.735145e-01  3.190076e-01  2.798753e-01  2.824763e-01
[46]  2.470766e-01  2.271602e-01  2.015906e-01  1.839331e-01  2.175831e-01
[51]  3.743969e-02  2.563874e-02  2.439735e-02  3.794740e-02  4.454647e-02
[56]  4.019509e-02  2.732245e-02  2.188961e-02  2.186576e-02  4.250931e-02
[61]  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
[66]  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
[71]  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
[76]  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
[81]  3.112704e-01  2.630389e-01  2.244922e-01  2.049814e-01  1.804736e-01
[86]  1.628496e-01  1.505630e-01  1.407512e-01  1.353575e-01  1.320533e-01

$eps
 [1]  0.0000000  1.2097343  0.5579901  0.1747780  0.1322100  2.4962135
 [7] -0.9567893 -3.3121444  0.3730446  1.4577289  0.0000000 -0.8726423
[13]  1.1557221 -2.7730501 -0.8930016  1.6661295  0.2734874  0.9902214
[19] -3.9340555 -1.4691469  0.0000000 -0.2863922  1.5648326 -0.3735751
[25] -0.5052128 -0.8036496 -0.3812639  0.9807474 -1.1003262  6.8949965

$loglik
[1] -40.02496

Fit model with different numbers of lags

In [4]:
print('With only 1 lag.')
scalar.bekk.fit(as.matrix(y[,2:4]),
                opts=list())[c('param','obj')]
print('With 3 lags')
scalar.bekk.fit(as.matrix(y[,2:4]),
                opts=list(lags = 3))[c('param','obj')]
print('With 5 lags')
scalar.bekk.fit(as.matrix(y[,2:4]),
                opts=list(lags = 5))[c('param','obj')]

[1] "With only 1 lag."


$param
     alpha       beta 
0.01959369 0.10338052 

$obj
[1] -18695.48

[1] "With 3 lags"


$param
     alpha       beta 
0.05796592 0.32341340 

$obj
[1] -24470.29

[1] "With 5 lags"


$param
    alpha      beta 
0.0635357 0.3464407 

$obj
[1] -23895.7

Fit model on a large amount of data and check performance with one lag

In [9]:
print("25 time-series with T=4025")
system.time(scalar.bekk.fit(as.matrix(y[,1:25]),opts=list(lags = 1))[c('param','obj')])
print("50 time-series with T=4025")
system.time(scalar.bekk.fit(as.matrix(y[,1:50]),opts=list(lags = 1))[c('param','obj')])
print("80 time-series with T=4026")
system.time(scalar.bekk.fit(as.matrix(y),opts=list(lags = 1))[c('param','obj')])

[1] "25 time-series with T=4025"


   user  system elapsed 
 13.298   0.941  14.254 

[1] "50 time-series with T=4025"


   user  system elapsed 
 82.648   3.975  86.909 

[1] "80 time-series with T=4026"


   user  system elapsed 
391.913  17.854 408.662 

Using more lags

In [10]:
print("25 time-series with T=4025 and three lags")
system.time(scalar.bekk.fit(as.matrix(y[,1:25]),opts=list(lags = 3))[c('param','obj')])
print("25 time-series with T=4025 and five lags")
system.time(scalar.bekk.fit(as.matrix(y[,1:25]),opts=list(lags = 5))[c('param','obj')])

[1] "25 time-series with T=4025 and three lags"


   user  system elapsed 
 22.309   1.008  23.373 

[1] "25 time-series with T=4025 and five lags"


   user  system elapsed 
 33.669   0.938  34.645 